In [2]:
ENV["PYTHONPATH"] = "/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";
D = 2;
using HDF5, PyPlot
using DelimitedFiles, PyPlot, LinearAlgebra, JLD2, PyCall, StaticArrays, Statistics
using Crystalline, Brillouin, MPBUtils, SymmetryBases
using Crystalline: TEST_αβγs, TEST_αβγ, dot, norm
using Crystalline: normscale, matching_littlegroups
using Crystalline: AbstractFourierLattice
topology_paper_dir = "../../TopologyPaper/"
include(topology_paper_dir * "get-freqs-symeigs.jl")
include(topology_paper_dir * "symeigs-from-io.jl");
include("../../Topology_ML/Phc_ML/fft.jl")
include("../get_uc_coefs.jl");

In [54]:
gidx = 4; 
dir = "./output/sg2/g$gidx/tm/";
filename_jld = "sg2-g$(gidx)-tm.jld2"
loaded_data = load(dir * filename_jld)

Dict{String, Any} with 4 entries:
  "symeigsdv"  => Dict{String, Vector{Vector{ComplexF64}}}[Dict("Y"=>[[1.0+0.0i…
  "summariesv" => Vector{BandSummary}[[1-band (trivial): [Y₁, B₁, A₁, Γ₁], 1-ba…
  "brs"        => BandRep[[0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 0, 1,…
  "lgirsd"     => Dict{String, Vector{LGIrrep{2}}}("Y"=>[LGIrrep{2}("Y₁", [1, 2…

In [55]:
summariesv = loaded_data["summariesv"];
first_band_summaries = [summaries[1] for summaries in summariesv];
first_band_topologies = [summary.topology for summary in first_band_summaries];

In [56]:
num_non_trivial = count(x -> x == NONTRIVIAL, first_band_topologies);
num_trivial = count(x -> x == TRIVIAL, first_band_topologies);
total = num_trivial + num_non_trivial;

In [57]:
num_non_trivial / total

0.2837

In [58]:
y_plus_idxs = findall(x->x[1] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);
y_minus_idxs = findall(x->x[2] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);

b_plus_idxs = findall(x->x[3] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);
b_minus_idxs = findall(x->x[4] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);

a_plus_idxs = findall(x->x[5] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);
a_minus_idxs = findall(x->x[6] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);

gamma_plus_idxs = findall(x->x[7] == 1 && x[end] == 1, [x.n for x in first_band_summaries]);
gamma_minus_idxs = findall(x->x[8] == 1 && x[end] == 1 , [x.n for x in first_band_summaries]);

In [59]:
f = h5open("sg2-data.h5", "r");

In [60]:
epsilon_Gs = [f["sg2/$(i)/epsilon_Gs-gidx=$(gidx)"][] for i in 1:10000];
epsilon_Gs_signs = [x ./abs.(x) for x in epsilon_Gs];

In [61]:
round.(100 * [count(x -> x[i] > 0,  epsilon_Gs_signs[y_plus_idxs]) for i in 2:10] / length(y_plus_idxs), digits=0)

9-element Vector{Float64}:
 100.0
  50.0
  50.0
  48.0
  49.0
  50.0
  49.0
  50.0
  49.0

In [62]:
round.(100 * [count(x -> x[i] >  0,  epsilon_Gs_signs[b_plus_idxs]) for i in 2:10] / length(b_plus_idxs), digits=0)

9-element Vector{Float64}:
  50.0
 100.0
  50.0
  49.0
  50.0
  50.0
  49.0
  51.0
  50.0

In [63]:
round.(100 * [count(x -> x[i] > 0,  epsilon_Gs_signs[a_plus_idxs]) for i in 2:10] / length(a_plus_idxs), digits=0)

9-element Vector{Float64}:
 50.0
 50.0
 68.0
 69.0
 50.0
 50.0
 49.0
 51.0
 50.0